In [ ]:
'''
This python code details the CRS of the data to be used in the ML codes (both in RF and ANN). 
This code is written by Saurav Dey Shuvo (saurav.met@gmail.com). 
'''

In [ ]:
# Libraries 
import os
import pygrib
import fiona
import rasterio
import netCDF4

# File paths
files = {
    "HRRR": "D:/2Courses/8902/data/hrrr/20231026/hrrr/20231001",
    "LandCover": "D:/2Courses/8902/data_model_build/LandCover/land_cover1.nc",
    "SlopeAspect": "D:/2Courses/8902/data_model_build/SlopeAspect/output_dem_slope_aspect1.nc",
    "DEM": "D:/2Courses/8902/data_model_build/DEM/DEM_bufferzone.tif",
    "NBR": "D:/2Courses/8902/data_model_build/NBR/MODIS_NBR_Nov2023.tif",
    "NDVI": "D:/2Courses/8902/data_model_build/NDVI/MODIS_NDVI_Nov2023.tif",
    "NDWI": "D:/2Courses/8902/data_model_build/NDWI/MODIS_NDWI_Nov2023.tif",
    "Shapefile": "D:/2Courses/8902/data/Filtered_circuits/Filtered_circuits/filtered_circuits.shp",
    "HainesIndex": "D:/2Courses/8902/data/hrrr/20231026/hrrr/20231001/haines_index_20231001.nc"
}

output_file = "crs_details.txt"

def get_crs_for_grib(grib_file):
    """
    Extract projection and grid information from a GRIB2 file using pygrib.
    """
    try:
        with pygrib.open(grib_file) as grib:
            crs_details = []
            for msg in grib:
                # Extract grid and projection information
                grid_type = msg.gridType
                proj = msg.projparams if hasattr(msg, 'projparams') else "Projection parameters not available"
                crs_details.append(f"Grid Type: {grid_type}, Projection: {proj}")
                # Break after first message (to avoid redundancy in most GRIB files)
                break
            return "\n".join(crs_details)
    except Exception as e:
        return f"Error processing GRIB2 file: {e}"

def process_hrrr_directory(hrrr_dir):
    """
    Process all GRIB2 files in a directory and extract CRS information.
    """
    crs_details = []
    for root, _, files_in_dir in os.walk(hrrr_dir):
        for file in files_in_dir:
            if file.endswith(".grib2"):
                grib_file = os.path.join(root, file)
                grib_crs = get_crs_for_grib(grib_file)
                crs_details.append(f"{file}: {grib_crs}")
    return "\n".join(crs_details) if crs_details else "No GRIB2 files found in the directory"

def get_crs_for_file(label, file_path):
    """
    General function to determine CRS for various file types.
    """
    try:
        if label == "HRRR":  # Process HRRR directory with pygrib
            return process_hrrr_directory(file_path)
        elif file_path.endswith(".nc"):  # For netCDF files
            with netCDF4.Dataset(file_path) as dataset:
                crs = None
                if "spatial_ref" in dataset.ncattrs():
                    crs = dataset.getncattr("spatial_ref")
                elif "crs" in dataset.ncattrs():
                    crs = dataset.getncattr("crs")
                else:
                    for var_name, var in dataset.variables.items():
                        if hasattr(var, "grid_mapping"):
                            crs = f"Grid mapping: {var.grid_mapping}"
                            break
                return crs if crs else "CRS attribute not found in netCDF"
        elif file_path.endswith(".tif"):  # For GeoTIFF files
            with rasterio.open(file_path) as src:
                return str(src.crs) if src.crs else "CRS not available for GeoTIFF"
        elif file_path.endswith(".shp"):  # For shapefiles
            with fiona.open(file_path, 'r') as shp:
                return str(shp.crs) if shp.crs else "CRS not available for Shapefile"
        else:
            return "File format not supported or unrecognized"
    except Exception as e:
        return f"Error processing file: {e}"

with open(output_file, "w") as f:
    for label, path in files.items():
        crs_details = get_crs_for_file(label, path)
        f.write(f"{label}:\n{crs_details}\n\n")

print(f"CRS details saved to {output_file}")
